<h1>Hello World</h1>


### Intro
Hey! Welcome to your first LangGraph lesson. We're gonna build a super simple 'Hello World' agent. It's basically a graph that takes a message and adds a greeting to it. Simple, right? Let's dive in!

### Imports
First things first, we need to import `StateGraph` which is the core of everything we build here, and some typing stuff to define what our data looks like.

In [ ]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph

### The State
Now, we need to define the **State**. Think of this as the memory of our agent. It holds the data that gets passed around between nodes. Here, our state just has one thing: a simple `message` string.

We also define a **Node**. A node is just a function that does some work. Our `greetingNode` takes the current state, adds 'Hey ' and ', how is your day going?' to the message, and returns the new state with the updated message.

In [ ]:
# AgentState
class AgentState(TypedDict): 
    # Our state schema - just a simple dictionary with a 'message' key
    message: str

# Node Schema
def greetingNode(state: AgentState) -> AgentState:
    """
    Simple node that adds a greeting message to the state
    """
    # formatting the message string
    state["message"] = "Hey " + state["message"] + ", how is your day going?"
    return state

### Building the Graph
Okay, now we build the graph!
1. Initialize the `StateGraph` with our `AgentState`.
2. Add our `greetingNode` to it using `add_node`.
3. Tell the graph where to start (`entry_point`) and where to stop (`finish_point`). Since we only have one node, it's both the start and the finish.
4. Finally, `compile()` it to create the runnable app.

In [ ]:
graph = StateGraph(AgentState)

graph.add_node("greeter", greetingNode) # Registering the node

graph.set_entry_point("greeter") # Start here
graph.set_finish_point("greeter") # End here

app = graph.compile() # Compile the graph into an app

### Visualization
Want to see what we built? LangGraph can visualize the graph for us. It's a great way to verify our logic.

In [ ]:
from IPython.display import Image, display
display(Image(app.get_graph().draw_mermaid_png()))

### Running It
Time to run it! We `invoke` the app with an initial state. Let's say the message is just "Bob".

In [ ]:
result = app.invoke({"message": "Bob"})

### The Result
Let's see the output. It should be the greeting we defined in the node.

In [ ]:
print(result["message"])